The <a href="https://data.cervantesvirtual.com">Biblioteca Virtual Miguel de Cervantes</a> published its catalogue as Linked Open data using as main vocabulary <a href="http://www.rdaregistry.info/">Resource, Description and Access</a>.

This example shows how to use <a href="https://www.w3.org/TR/sparql11-query/">SPARQL</a> as a query language in Linked Open Data repositories.

### First of all, we init the SPARQLWrapper service with the SPARQL endpoint

In [31]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper(
    "https://data.cervantesvirtual.com/sparql"
)
sparql.setReturnFormat(JSON)
sparql.addDefaultGraph("http://localhost:8890/DAV/bvmc")

### Let's write our first SPARQL sentence

We could be interested in the classes used in the repository

In [32]:
sparql.setQuery("""
    SELECT DISTINCT ?class
    WHERE {
        ?s a ?class . filter(regex(str(?class), 'rdaregistry'))
    }
    LIMIT 10
    """
)

### Now we retrieve the results

In [33]:
try:
    ret = sparql.queryAndConvert()

    for r in ret["results"]["bindings"]:
        print('Class: ', r['class']['value'])
except Exception as e:
    print(e)

Class:  http://rdaregistry.info/Elements/c/Person
Class:  http://rdaregistry.info/Elements/c/CorporateBody
Class:  http://rdaregistry.info/Elements/c/Family
Class:  http://rdaregistry.info/Elements/c/Expression
Class:  http://rdaregistry.info/Elements/c/Item
Class:  http://rdaregistry.info/Elements/c/Manifestation
Class:  http://rdaregistry.info/Elements/c/Work


### We can also ask for metadata about the resources

In the following example, we retrieve the metadata about the work ["El ingenioso hidalgo Don Quijote de la Mancha"](https://data.cervantesvirtual.com/work/2904).

In [34]:
sparql.setQuery("""
    PREFIX rdac: <http://rdaregistry.info/Elements/c/>
    PREFIX rdaw: <http://rdaregistry.info/Elements/w/>
    PREFIX rdaa: <http://rdaregistry.info/Elements/a/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>

    SELECT DISTINCT ?title ?author ?name ?subject
    WHERE {
        <https://data.cervantesvirtual.com/work/2904> a rdac:Work .
        <https://data.cervantesvirtual.com/work/2904> rdaw:titleOfTheWork ?title .
        <https://data.cervantesvirtual.com/work/2904> dc:subject ?subject .
        <https://data.cervantesvirtual.com/work/2904> rdaw:author ?author .
        ?author rdaa:nameOfThePerson ?name
    }
    LIMIT 10
    """
)

### And we can check the results

In [35]:
try:
    ret = sparql.queryAndConvert()

    for r in ret["results"]["bindings"]:
        print('Title: ', r['title']['value'])
        print('Subject: ', r['subject']['value'])
        print('Author: ', r['author']['value'])
        print('Name: ', r['name']['value'])
except Exception as e:
    print(e)

Title:  El ingenioso hidalgo Don Quijote de la Mancha
Subject:  Novela española -- Siglo 17º
Author:  https://data.cervantesvirtual.com/person/40
Name:  Cervantes Saavedra, Miguel de


### Let's use the information from Wikidata

We can retrieve the characters which appear in a particular item from Wikidata. The property wdt:P3976 identifies works in Wikidata from the Biblioteca Virtual Miguel de Cervantes and the property wdt:P674 is used to provide characters that appears in a resource.

In [36]:
sparql = SPARQLWrapper(
    "https://query.wikidata.org/sparql"
)
sparql.setReturnFormat(JSON)

sparql.setQuery("""
    PREFIX rdac: <http://rdaregistry.info/Elements/c/>
    PREFIX rdaw: <http://rdaregistry.info/Elements/w/>
    PREFIX rdaa: <http://rdaregistry.info/Elements/a/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>

    SELECT DISTINCT ?character ?characterLabel
    WHERE {
        ?work wdt:P3976 "2904".
        ?work wdt:P674 ?character . 
        SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],es". }
    }
    LIMIT 50
    """
)

### And now we see the results

In [37]:
try:
    ret = sparql.queryAndConvert()

    for r in ret["results"]["bindings"]:
        print('Character: ', r['character']['value'])
        print('Character: ', r['characterLabel']['value'])
        print('*****')
except Exception as e:
    print(e)

Character:  http://www.wikidata.org/entity/Q5129635
Character:  Clavileño
*****
Character:  http://www.wikidata.org/entity/Q6120178
Character:  Sansón Carrasco
*****
Character:  http://www.wikidata.org/entity/Q6072772
Character:  Pero Pérez
*****
Character:  http://www.wikidata.org/entity/Q5563337
Character:  Ginés de Pasamonte
*****
Character:  http://www.wikidata.org/entity/Q6113413
Character:  el rucio
*****
Character:  http://www.wikidata.org/entity/Q23807432
Character:  Maese Nicolás
*****
Character:  http://www.wikidata.org/entity/Q52733938
Character:  Marcela
*****
Character:  http://www.wikidata.org/entity/Q56419457
Character:  Don Fernando
*****
Character:  http://www.wikidata.org/entity/Q21401597
Character:  Maritornes
*****
Character:  http://www.wikidata.org/entity/Q630823
Character:  Sancho Panza
*****
Character:  http://www.wikidata.org/entity/Q1918252
Character:  Rocinante
*****
Character:  http://www.wikidata.org/entity/Q52734852
Character:  Dorotea
*****
Character:  ht

We could also retrieve the images of the characters:

In [38]:
sparql = SPARQLWrapper(
    "https://query.wikidata.org/sparql"
)
sparql.setReturnFormat(JSON)

sparql.setQuery("""
    SELECT DISTINCT ?image ?characterLabel
    WHERE {
        ?work wdt:P3976 "2904".
        ?work wdt:P674 ?character . 
        ?character wdt:P18 ?image .
        SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],es". }
    }
    LIMIT 5
    """
)

In [39]:
from IPython.display import display, HTML

try:
    ret = sparql.queryAndConvert()

    for r in ret["results"]["bindings"]:
        print('Character: ', r['characterLabel']['value'])
        print('Image: ', r['image']['value'])
        display(HTML("<img width='100px' src='"+ r['image']['value'] +"'>"))
except Exception as e:
    print(e)

Character:  Sancho Panza
Image:  http://commons.wikimedia.org/wiki/Special:FilePath/1879%2C%20El%20ingenioso%20hidalgo%20D.%20Quijote%20de%20la%20Mancha%2C%20Sancho%20Panza%2C%20Mestres%20%28cropped%29.jpg


Character:  Anselmo
Image:  http://commons.wikimedia.org/wiki/Special:FilePath/202%20084%20quijote%20cap33%20%28cropped%29%20Anselmo.jpg


Character:  Rocinante
Image:  http://commons.wikimedia.org/wiki/Special:FilePath/CavalloChishotte.jpg


Character:  Maritornes
Image:  http://commons.wikimedia.org/wiki/Special:FilePath/Don%20Quijote%2C%20Luis%20Tasso%2C%20%281894%3F%29%20%22El%20lecho%2C%20que%20era%20un%20poco%20endeble...d%C3%ADo%20consigo%20en%20el%20suelo...%22%20%285789341875%29%20%28cropped%29%20Maritornes.jpg


Character:  Dorotea
Image:  http://commons.wikimedia.org/wiki/Special:FilePath/1905-05-27%2C%20Blanco%20y%20Negro%2C%20Dorotea%2C%20Cecilio%20Pla%20%28cropped%29.jpg
